In [ ]:
import os
import subprocess
import time
import shutil
import pandas as pd

# Compile C++ code
print("Compiling tree_packer.cpp...")
subprocess.run(["g++", "-O3", "-march=native", "-std=c++17", "-fopenmp", "-o", "tree_packer.exe", "tree_packer.cpp"], check=True)

print("Compiling bp.cpp...")
subprocess.run(["g++", "-O3", "-std=c++17", "-o", "bp.exe", "bp.cpp"], check=True)

print("Compilation successful.")

In [ ]:
# Configuration
os.environ['OMP_NUM_THREADS'] = '4' # Conservative estimate
n_steps = 1000 # Reduced for first run to test
r_restarts = 4

# Run Tree Packer
print("Running Tree Packer...")
cmd = ["./tree_packer.exe", "-n", str(n_steps), "-r", str(r_restarts)]

process = subprocess.Popen(
    cmd,
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    text=True,
    bufsize=1,
    universal_newlines=True
)

for line in process.stdout:
    print(line, end='')

process.wait()

if process.returncode != 0:
    print("Tree Packer failed.")
else:
    print("Tree Packer finished.")

In [ ]:
# Check for output file
# The reference notebook expects submission_v21.csv
output_file = "submission_v21.csv"
if os.path.exists(output_file):
    print(f"Found {output_file}")
    shutil.copy(output_file, "submission.csv")
else:
    print(f"{output_file} not found. Checking directory...")
    print(os.listdir("."))
    # If not found, maybe it outputs to stdout or a different file?
    # I'll check if sample_submission can be used as fallback for BP
    if not os.path.exists("submission.csv"):
        print("Using sample_submission.csv as fallback")
        shutil.copy("/home/data/sample_submission.csv", "submission.csv")

In [ ]:
# Run Backward Propagation
print("Running Backward Propagation...")
if os.path.exists("submission.csv"):
    cmd_bp = ["./bp.exe", "submission.csv", "submission_bp.csv"]

    process_bp = subprocess.Popen(
        cmd_bp,
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        text=True,
        bufsize=1,
        universal_newlines=True
    )

    for line in process_bp.stdout:
        print(line, end='')

    process_bp.wait()

    if process_bp.returncode == 0:
        print("BP finished.")
        if os.path.exists("submission_bp.csv"):
            shutil.copy("submission_bp.csv", "submission.csv")
    else:
        print("BP failed.")
else:
    print("No submission.csv found to optimize.")